## Where to Read More
This notebook will help me keep track of my experimental weighted crypto portfolio that I started for this year (2022). 

You can read more about the parameters and qualities of the portfolio in the [original blog post](https://andresberejnoi.com/best-cryptos-to-invest-in-2022/) I wrote. I write an update every month, which you can follow on my blog.

You can check my Youtube video about using the [Coinbase Pro API here](https://www.youtube.com/watch?v=a74pQbHgdXw). It's just 9 minutes long and covers a good amount of the basic actions.

### Crypto Market Data
At the moment, I'm making use of Coinbase Pro API through the [cbpro library](https://github.com/danpaquin/coinbasepro-python) to collect market data of the coins I'm tracking. The endpoint I'll be using is `https://api.exchange.coinbase.com/products/{product_id}/candles`, and the documentation can be found [here](https://docs.cloud.coinbase.com/exchange/reference/exchangerestapi_getproductcandles).

You can also use other APIs to gather the data. I was initially using AlphaVantage with an API key, but Coinbase Pro supported more of the coins I'm tracking. 
The endpoint I would use for AlphaVantage is `DIGITAL_CURRENCY_DAILY `, and the documentation can be found [here](https://www.alphavantage.co/documentation/#currency-daily). Additionally, [this csv](https://www.alphavantage.co/digital_currency_list/) file from AlphaVantage lists all the cryptocurrencies supported by the API.


### AlphaVantage Helper Function

In [1]:
def get_daily_data(symbol, api_key, start_date=None, end_date=None, quote_currency='USD', simplify=True):
    '''Uses AlphaVantage endpoint DIGITAL_CURRENCY_DAILY to return a dataframe.
    I created this function to simplify the process'''
    
    _base_url    = "https://www.alphavantage.co/"
    api_function = 'DIGITAL_CURRENCY_DAILY'

    url = f'{_base_url}query?function={api_function}&symbol={symbol}&market={quote_currency}&apikey={api_key}'
    r = requests.get(url)
    data = r.json()
    
    #check that we received proper data
    if 'Error Message' in data:
        print(f"\t--> Error retrieving {symbol}. It's probably not supported.")
        print("\t    Returning `None`")
        return None
    
    #we need to parse data.
    _candle_data = data["Time Series (Digital Currency Daily)"]
    df = pd.DataFrame(_candle_data).transpose()  #without transpose, we get each daily candle as a column. We want each candle as a row, and each column as the OHLC data
    
    #--convert index to datetime
    df.index = pd.to_datetime(df.index)
    
    #AlphaVantage returns data both in USD and whatever quote currency we asked for. 
    # in case we do not want repetition, simplify will remove the fixed USD market and return a clearner dataframe just with the requested quote currency
    if simplify:
        desired_columns = ['1a. open (USD)','2a. high (USD)','3a. low (USD)','4a. close (USD)','5. volume','6. market cap (USD)']
        new_names       = ['open','high','low','close','volume','market_cap']
        
        dict_replacement = {old_name:new_name for old_name, new_name in zip(desired_columns, new_names)}
        df = df[desired_columns]
        
        df = df.rename(columns=dict_replacement)
    
    #-- in case start and end dates are specified:
    if start_date:
        if end_date:
            return df[start_date:end_date]
        else:
            return df[start_date:]
    elif end_date:
        return df[:end_date]
    
    return df

## Imports And Setup

In [2]:
import os
import pandas as pd
import requests
import time
import cbpro

from api_keys import alpha_vantage_key   #if you use AlphaVantage, put your own key here or in a different file

In [3]:
_data_folder = "./data/"
list_coins = ['BTC', 'ETH', 'BNB', 'SOL', 'ADA', 'WLUNA', 'AVAX', 'DOT', 'DOGE',
              'SHIB', 'MATIC', 'CRO', 'LTC', 'UNI', 'LINK', 'ALGO', 'BCH', 'XLM',
              'ATOM', 'AXS']

## Using Coinbase Pro Library to Get Historic Data
For the `get_historic_rates` function, the granularity for daily candles is 86400. Check the documentation [here](https://docs.cloud.coinbase.com/exchange/reference/exchangerestapi_getproductcandles).

In [4]:
start_date = '2021-12-23'
end_date   = '2022-03-31'

client = cbpro.PublicClient()

In [5]:
coinbase_data_individual_dfs = {}
quote_currency = 'USD'

for coin in list_coins:
    pair = f"{coin}-{quote_currency}"
    
    #cbpro will return a list of lists. We need to turn it into a dataframe
    columns = ['date','open','high','low','close','volume']
    nested_list_data = client.get_product_historic_rates(pair, start=start_date, end=end_date, granularity=86400)
    
    coinbase_data_individual_dfs[coin] = pd.DataFrame(nested_list_data, columns=columns).set_index('date',drop=True).sort_index(ascending=True)
    
    #convert date from unix time to iso format (easier to look at)
    coinbase_data_individual_dfs[coin].index = pd.to_datetime(
        coinbase_data_individual_dfs[coin].index,
        origin = 'unix',
        unit='s')

In [6]:
coinbase_data_individual_dfs['BTC']

,open,high,low,close,volume
date,,,,,
2021-12-23,48032.16,51397.82,48608.61,50842.20,16627.046280
2021-12-24,50445.55,51878.60,50842.06,50851.38,11690.454182
2021-12-25,50191.84,51171.68,50852.31,50428.31,5550.571532
2021-12-26,49460.00,51295.33,50428.31,50801.79,6863.394738
2021-12-27,50480.00,52100.00,50804.33,50717.77,11596.914682
...,...,...,...,...,...
2022-03-27,44437.22,46950.00,44538.21,46850.01,10356.103170
2022-03-28,46662.28,48240.00,46850.01,47144.92,18787.663187
2022-03-29,46589.00,48124.94,47146.92,47454.19,15776.651086


### Save Full Candle Files For Each Coin
Before saving, the index is sorted so that each candle is in ascending order (ascending towards the bottom).

* ***WARNING***:
Coinbase Pro returns empty data for Binance coin (BNB). That means that we need to fix the file or dataframe before it is saved. I will use AlphaVantage API to do this since I already wrote some code for it in this notebook.

In [7]:
daily_close_coins = {}   #store only the closing data. This will be turned into a dataframe later
for i, coin in enumerate(coinbase_data_individual_dfs):
    pair = f"{coin}_{quote_currency}"
    _output_file = f"{pair}_daily_candles.csv"
    output_file = os.path.join(_data_folder, _output_file)
    
    print(f"--> Saving candles for pair: {pair}\n\tat {output_file}\n")
    
    _df = coinbase_data_individual_dfs[coin]
    
    # collect closing data
    daily_close_coins[coin] = _df['close']
    
    # write dataframe to file
    _df.to_csv(output_file, 
               index=True,
               mode='w',  #python file mode. 'a' is for append, 'w' for write, etc
               #header=False, #use this when appending, so column names are not written in the middle of the file
               )
    

--> Saving candles for pair: BTC_USD
	at ./data/BTC_USD_daily_candles.csv

--> Saving candles for pair: ETH_USD
	at ./data/ETH_USD_daily_candles.csv

--> Saving candles for pair: BNB_USD
	at ./data/BNB_USD_daily_candles.csv

--> Saving candles for pair: SOL_USD
	at ./data/SOL_USD_daily_candles.csv

--> Saving candles for pair: ADA_USD
	at ./data/ADA_USD_daily_candles.csv

--> Saving candles for pair: WLUNA_USD
	at ./data/WLUNA_USD_daily_candles.csv

--> Saving candles for pair: AVAX_USD
	at ./data/AVAX_USD_daily_candles.csv

--> Saving candles for pair: DOT_USD
	at ./data/DOT_USD_daily_candles.csv

--> Saving candles for pair: DOGE_USD
	at ./data/DOGE_USD_daily_candles.csv

--> Saving candles for pair: SHIB_USD
	at ./data/SHIB_USD_daily_candles.csv

--> Saving candles for pair: MATIC_USD
	at ./data/MATIC_USD_daily_candles.csv

--> Saving candles for pair: CRO_USD
	at ./data/CRO_USD_daily_candles.csv

--> Saving candles for pair: LTC_USD
	at ./data/LTC_USD_daily_candles.csv

--> Saving 

### Fix Missing BNB Data With AlphaVantage

In [8]:
bnb_df = get_daily_data('BNB',alpha_vantage_key, start_date=start_date, end_date=end_date, quote_currency='USD', simplify=True)

In [9]:
daily_close_coins['BNB'] = bnb_df['close']  #get only closing data

In [10]:
_output_bnb_file = os.path.join(_data_folder, f"BNB_{quote_currency}_daily_candles.csv")
bnb_df.to_csv(_output_bnb_file, index_label='date', mode='a')

In [11]:
df_coinbase = pd.DataFrame(daily_close_coins)
df_coinbase

,BTC,ETH,BNB,SOL,ADA,WLUNA,AVAX,DOT,DOGE,SHIB,MATIC,CRO,LTC,UNI,LINK,ALGO,BCH,XLM,ATOM,AXS
2021-12-23,50842.20,4113.08,549.00000000,190.07,1.4743,94.42,121.37,29.17,0.1851,0.000039,2.6858,0.5951,163.74,18.18,22.19,1.4697,455.93,0.287776,28.38,107.47
2021-12-24,50851.38,4049.65,542.10000000,190.65,1.3935,95.77,114.93,28.17,0.1862,0.000036,2.4687,0.6280,161.35,17.50,21.47,1.5760,452.27,0.278628,26.87,104.62
2021-12-25,50428.31,4097.43,547.40000000,193.06,1.4542,98.39,114.86,28.86,0.1911,0.000038,2.6409,0.6434,158.00,17.49,22.12,1.5748,455.36,0.290047,29.79,109.27
2021-12-26,50801.79,4065.91,546.40000000,198.00,1.4549,99.63,115.07,31.34,0.1899,0.000038,2.8793,0.6233,155.96,18.67,22.99,1.6079,451.86,0.291987,32.17,107.62
2021-12-27,50717.77,4038.64,563.70000000,195.74,1.5164,90.88,113.96,30.97,0.1879,0.000039,2.7267,0.6234,155.78,19.11,23.02,1.6631,465.45,0.300288,29.68,107.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-27,46850.01,3296.29,430.50000000,107.09,1.1845,94.18,89.91,22.48,0.1447,0.000026,1.6949,0.4865,129.68,11.40,16.87,0.9498,376.78,0.229866,29.95,68.29
2022-03-28,47144.92,3334.47,430.20000000,105.84,1.1694,97.25,89.58,21.97,0.1425,0.000027,1.6575,0.4802,127.59,10.91,16.51,0.9075,368.10,0.229783,29.36,66.44
2022-03-29,47454.19,3401.71,434.20000000,111.64,1.1909,109.03,92.79,22.44,0.1444,0.000027,1.6760,0.4805,130.08,11.25,16.99,0.9232,372.34,0.233124,30.24,64.18
2022-03-30,47078.03,3386.68,442.10000000,120.79,1.1906,106.04,95.57,22.46,0.1432,0.000027,1.6827,0.4737,131.12,11.74,17.23,0.9268,380.13,0.234742,30.02,64.42


### Save Single File With Only Closing Data For All Coins Combined

In [12]:
close_candle_output_file = os.path.join(_data_folder, "2022_portfolio_daily_close.csv")
df_coinbase.to_csv(close_candle_output_file,
                   index=True,
                   index_label='date',
                   mode='w',  #python file mode. 'a' is for append, 'w' for write, etc
                   #header=False, #use this when appending, so column names are not written in the middle of the file
                  )

---
---

## Retrieving Historic Data With API From ALPHA VANTAGE
The AlphaVantage section below is going to be turned into markdown so I don't accidentally run it. If you want to use it, simply turn those cells into code cells.

start_date = "2022-01-01"
end_date   = "2022-02-28"

daily_close_candles = {}
problematic_coins = []  #used to collect coins that could not be found in AlphaVantage API

for coin in df['Coin']:
    print(f"--> Parsing data for {coin}")
    
    if coin in daily_close_candles:  #skip coins that are already in the candle dict
        continue
        
    if coin=='WLUNA':
        coin = 'LUNA'  #alphavantage does not support WLUNA, so we used the real token
        print("\t* Changed WLUNA to LUNA")
    candles = get_daily_data(symbol         = coin,
                             api_key        = alpha_vantage_key,
                             start_date     = start_date,
                             end_date       = end_date,
                             quote_currency = 'USD',
                             simplify       = True)
    
    if candles is None:
        problematic_coins.append(coin)
        continue
        
    close_candles = candles['close']  # I only want the closing data
    daily_close_candles[coin] = close_candles
    
    time.sleep(13)  #AlphaVantage allows 5 requests per minute, so I'm delaying between requests

portfolio_df = pd.DataFrame(daily_close_candles)
portfolio_df

output_file = os.path.join(_data_folder, "portfolio_daily_close.csv")
portfolio_df.to_csv(output_file)